In [11]:
import pandas as pd

train_df = pd.read_table('data-input/indeed_ml_dataset/train.tsv')
train_df = train_df.fillna('')
train_df.head(5)

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [12]:
possible_tags = ['part-time-job', 'full-time-job', 'hourly-wage', 'salary', 
                 'associate-needed', 'bs-degree-needed', 'ms-or-phd-needed',
                 'licence-needed', '1-year-experience-needed', '2-4-years-experience-needed',
                 '5-plus-years-experience-needed', 'supervising-job']

Y = []
for tags_str in train_df.iloc[:, 0]:
    tags = tags_str.split(' ')
    
    tags_vector = [0] * len(possible_tags)
    for idx, possible_tag in enumerate(possible_tags):
        if possible_tag in tags:
            tags_vector[idx] = 1
    Y.append(tags_vector)
        

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

descriptions = train_df.iloc[:, 1]
X_train = tfidf.fit_transform(descriptions)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

Y = np.array(Y)

lr = LogisticRegression()
ovr = OneVsRestClassifier(lr)
ovr.fit(X_train,Y)
print(ovr.multilabel_)

True


In [15]:
test_df = pd.read_table('data-input/indeed_ml_dataset/test.tsv')

descriptions = test_df.iloc[:, 0]
X_test = tfidf.transform(descriptions)
print(X_test.shape)

(2921, 408403)


In [16]:
Y_pred = ovr.predict(X_test)
print(Y_pred.shape)

(2921, 12)


In [17]:
output_file = open('data-output/tags.tsv', 'w')
output_file.write('tags\n')

for tags_vector in Y_pred:
    tags_arr = np.array(possible_tags)[tags_vector.nonzero()]
    tags_list = tags_arr.tolist()
    output_file.write(' '.join(tags_list) + '\n')

output_file.close()